In [ ]:
from datetime import datetime, timedelta
import pickle
from flask import Blueprint, render_template, request, jsonify
import pandas as pd
import numpy as np
import pymysql
from model.content_based_recommender import ContentBasedRecommender


# ------------------- Flask Blueprint ------------------- #


survey = Blueprint("survey", __name__, template_folder="templates")


# ------------------- GETTING TODAY'S YEAR/MONTH/DATE ------------------- #


today = datetime.strftime(datetime.now() - timedelta(1), '%Y%m%d')  # 오늘
yesterday = datetime.strftime(datetime.now() - timedelta(2), '%Y%m%d')  # 어제
today_year = today[:4]  # 이번년도


# ------------------- SURVEY PREDICT MODEL ------------------- #


# with open('content_based_recommender.pkl', 'rb') as f:
#     model = pickle.load(f)  # 단 한줄씩 읽어옴


# ------------------- SURVEY PAGE API ------------------- #


@survey.route("/")  # Survey page
def survey_page():
    return render_template("survey.html")


@survey.route("/result", methods=["GET"])  # Receiving survey answer
def survey_answer():
    try:
        db = pymysql.connect(host='abandoned-dogs.cdlurfzj5gl4.ap-northeast-2.rds.amazonaws.com',
                             port=3306, user='kaist', passwd='0916', db='abandoned_dog', charset="utf8")
        cursor = db.cursor()

        sur_res = request.args  # Stores user's survey answer
        user_answer = {
            'user_age': sur_res.get('user_age'),
            'user_sex': sur_res.get('user_sex'),
            'user_house_type': sur_res.get('user_house_type'),
            'dog_experience': {
                'dog_experience_yn': sur_res.get('dog_experience'),
                'dog_num': int(sur_res.get('dog_num', 0)),
                'dog_time': int(sur_res.get('dog_time', 0))
            },
            'dog_num': int(sur_res.get('dog_num', 0)),
            'dog_time': int(sur_res.get('dog_time', 0)),
            'user_family_size': sur_res.get('user_family_size'),
            'neighbor_agreement': sur_res.get('neighbor_agreement'),
            'user_kids': sur_res.get('user_kids'),
            'dog_size': sur_res.get('dog_size'),
            'shedding_level': int(sur_res.get('shedding_level')),
            'bark_tolerance': int(sur_res.get('bark_tolerance')),
            'spend_time': sur_res.get('spend_time'),
            'spend_type': sur_res.get('spend_type'),
            'dog_sex': sur_res.get('dog_sex'),
            'dog_environment': sur_res.get('dog_environment'),
            'dog_support_agreement': sur_res.get('dog_support_agreement'),
            'dog_health_agreement': sur_res.get('dog_health_agreement'),
            'want_dog_age': sur_res.get('want_dog_age'),
            'neuter_yn': sur_res.get('neuter_yn'),
            'user_id': '1'
        }

        query = 'SELECT * FROM mixprinted_include'
        panel_info = pd.read_sql(sql=query, con=db)

        query = 'SELECT * FROM breeds_panel'
        breeds_panel = pd.read_sql(sql=query, con=db)

        query = 'SELECT * FROM adopter_data'
        adopter_data = pd.read_sql(sql=query, con=db)

        query = 'SELECT * FROM dog_list ' \
                'WHERE processState="보호중" AND kindCd != "믹스견"'
        dog_list_data = pd.read_sql(sql=query, con=db)

        query = 'SELECT * FROM breed_info'
        breed_info = pd.read_sql(sql=query, con=db)
        #user_survey_data = {key: 3 for key in survey_lst}

        recommender = ContentBasedRecommender(breeds_panel=breeds_panel,
                                              target_user_survey=user_answer,
                                              adopter_data=adopter_data,
                                              dog_list_data=dog_list_data,
                                              breed_info=breed_info,
                                              panel_info=panel_info)

        # #print(user)
        recommender.fit_transform(target_user_survey=user_answer)

        with open(file='model/content_based_recommender.pkl', mode='wb') as f:
            pickle.dump(recommender, f)

        recommended_dogs, recommended_scores = recommender.predict(
            user_survey_data=user_answer)

        survey_to_data = recommender.get_processed_user_data()
        dog_data = recommender.get_processed_dog_data()
        dog_data_dict = {}
        for dog_dict in dog_data:
            dog_data_dict[list(dog_dict.keys())[0]
                          ] = dog_dict[list(dog_dict.keys())[0]]

        # len(recommended_dogs) : 공고번호의 수 (Int)
        # recommended_dogs : 내림차순 정렬된 공고번호 (List)
        # recommended_scores : 각 유사도 스코어 Percentage (List)
        # survey_to_data : 유저 설문조사 수치로 바꾼 딕셔너리 (Dict)
        # dog_data : recommended_dogs의 공고번호의 개들 수치화 (List[Dict])

        df = pd.DataFrame(dog_data, columns=dog_data.keys())
        

        ranking_order = []
        for i in range(len(recommended_dogs)):
            # sql = f"SELECT popfile, kindCd, sexCd, happenDt, noticeNo, processState FROM dog_list WHERE desertionNo = '{recommended_dogs[i]}';"
            # cursor.execute(sql)
            # result = cursor.fetchall()

            # for dog_info in result:
            #     popfile = dog_info[0]
            #     kindCd = dog_info[1]
            #     sexCd = dog_info[2]
            #     happenDt = dog_info[3][:4] + "/" + \
            #         dog_info[3][4:6] + "/" + dog_info[3][6:]
            #     noticeNo = dog_info[4].replace("-", " ")[:5]
            #     processState = dog_info[5]

            survey_res = {
                "des_no": recommended_dogs[i],  # Desertion No
                # Recommend Score (Percentage)
                "rec_score": "{:.2%}".format(recommended_scores[i]),
                # Panel data trait
                "trait_score": dog_data_dict[recommended_dogs[i]],
                "popfile": popfile,
                "kindCd": kindCd,
                "sexCd": sexCd,
                "happenDt": happenDt,
                "noticeNo": noticeNo,
                "processState": processState
            }
            ranking_order.append(survey_res)

        return render_template("survey_result.html", ranking_order=ranking_order)
        # return jsonify(survey_results)

    # except Exception as e:
    #     print("/survey/result ERROR")
    #     print(e)

    finally:
        cursor.close()
        db.close()


In [9]:
from datetime import datetime, timedelta
import pickle
from flask import Blueprint, render_template, request, jsonify
import pandas as pd
import numpy as np
#import pymysql
from content_based_recommender import ContentBasedRecommender


ModuleNotFoundError: No module named 'model'

In [10]:
!conda activate Abandoned-Dog-Project-Web

In [4]:
!pip list

Package            Version
------------------ ---------
astroid            2.11.7
autopep8           1.6.0
certifi            2022.6.15
cffi               1.15.1
charset-normalizer 2.1.1
click              8.0.4
colorama           0.4.5
cramjam            2.5.0
cryptography       37.0.1
dill               0.3.5.1
fastparquet        0.8.2
Flask              2.1.3
fsspec             2022.7.1
idna               3.3
importlib-metadata 4.11.3
isort              5.9.3
itsdangerous       2.0.1
Jinja2             3.0.3
joblib             1.1.0
lazy-object-proxy  1.6.0
MarkupSafe         2.1.1
mccabe             0.7.0
mypy               0.910
mypy-extensions    0.4.3
numpy              1.23.2
packaging          21.3
pandas             1.4.3
pip                22.1.2
platformdirs       2.4.0
psutil             5.9.0
pyarrow            9.0.0
pycodestyle        2.8.0
pycparser          2.21
pylint             2.14.5
PyMySQL            1.0.2
pyparsing          3.0.9
python-dateutil    2.8.2
pytz   